<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
!pip install yfinance
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [75]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [76]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [77]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'GODREJIND', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK',
       'ICICIPRULI', 'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI',
       'MCDOWELL-N', 'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM',
       'NAUKRI', 'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN',
       'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL',
       'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'],
      dtype=object)

In [78]:
mypf['InPortfolio'].value_counts()

0    48
1    13
Name: InPortfolio, dtype: int64

In [79]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

1060.0

In [80]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [81]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['200_EMA_Lag10'] = stock_df['200_EMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_EMA'] - stock_df['200_EMA_Lag10'])*100/stock_df['200_EMA'],2)
    stock_df['50_EMA_Lag5'] = stock_df['50_EMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_EMA'] - stock_df['50_EMA_Lag5'])*100/stock_df['50_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    stock_df.drop(['20_EMA','50_EMA','200_EMA','200_EMA_Lag10','50_EMA_Lag5'], axis=1, inplace=True)
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [82]:
stock_prec_dev('^NSEI').tail(1)

[*********************100%***********************]  1 of 1 completed


,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%
Date,,,,,,,,,,
2022-12-06,18608.0,63.0,^NSEI,7.26,0.76,0.82,18812.0,15294.0,94.2,5.8


In [83]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [84]:
# heavy upside
df_prec_dev.sort_values(by = 'Upside%', ascending=False).head(20)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-06,669.0,41.0,MOTILALOFS.NS,-13.66,-1.55,-1.01,980.0,654.0,4.65,95.35,930.0,39.01
1,2022-12-06,463.0,72.0,GODREJIND.NS,-0.86,-0.64,1.15,644.0,405.0,24.27,75.73,638.0,37.80
2,2022-12-06,473.0,43.0,ICICIPRULI.NS,-10.18,-1.14,-1.00,616.0,447.0,15.59,84.41,640.0,35.31
3,2022-12-06,1287.0,61.0,SBILIFE.NS,6.58,0.50,0.56,1332.0,1025.0,85.49,14.51,1604.0,24.63
4,2022-12-06,6707.0,37.0,BAJFINANCE.NS,-2.22,-0.20,-0.82,7853.0,5284.0,55.41,44.59,8015.0,19.50
5,2022-12-06,923.0,51.0,ICICIBANK.NS,11.64,1.33,0.77,953.0,654.0,90.07,9.93,1094.0,18.53
6,2022-12-06,897.0,65.0,AXISBANK.NS,17.16,1.70,1.42,915.0,626.0,93.93,6.07,1060.0,18.17
7,2022-12-06,551.0,42.0,JUBLFOOD.NS,-7.02,-0.84,-1.04,786.0,464.0,27.13,72.87,649.0,17.79
8,2022-12-06,3290.0,33.0,EICHERMOT.NS,4.91,0.86,-0.60,3850.0,2256.0,64.87,35.13,3865.0,17.48
9,2022-12-06,1256.0,46.0,M&M.NS,11.21,1.15,0.08,1354.0,691.0,85.16,14.84,1468.0,16.88


In [85]:
# buy or accumulate (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=True).head(20)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-06,669.0,41.0,MOTILALOFS.NS,-13.66,-1.55,-1.01,980.0,654.0,4.65,95.35,930.0,39.01
36,2022-12-06,3360.0,44.0,DIVISLAB.NS,-11.76,-1.23,-0.63,4678.0,3266.0,6.66,93.34,3560.0,5.95
2,2022-12-06,473.0,43.0,ICICIPRULI.NS,-10.18,-1.14,-1.00,616.0,447.0,15.59,84.41,640.0,35.31
44,2022-12-06,410.0,64.0,WIPRO.NS,-9.78,-1.32,0.50,722.0,375.0,10.23,89.77,425.0,3.66
7,2022-12-06,551.0,42.0,JUBLFOOD.NS,-7.02,-0.84,-1.04,786.0,464.0,27.13,72.87,649.0,17.79
31,2022-12-06,1085.0,56.0,TECHM.NS,-6.70,-0.77,0.66,1806.0,965.0,14.27,85.73,1158.0,6.73
12,2022-12-06,78.0,63.0,NATIONALUM.NS,-5.72,-1.20,1.33,130.0,67.0,17.86,82.14,90.0,15.38
14,2022-12-06,4057.0,56.0,NAUKRI.NS,-4.30,-0.71,0.25,5740.0,3375.0,28.82,71.18,4677.0,15.28
39,2022-12-06,619.0,55.0,BERGEPAINT.NS,-3.70,-0.31,0.16,791.0,561.0,25.30,74.70,651.0,5.17
29,2022-12-06,4006.0,44.0,DMART.NS,-3.69,-0.48,-0.63,4821.0,3231.0,48.77,51.23,4282.0,6.89


In [86]:
# sell or downsize (based on std dev from 200 EMA)
df_prec_dev.sort_values(by = 'Dev%_200', ascending=False).head(15)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
55,2022-12-06,322.0,66.0,CANBK.NS,31.76,3.69,2.44,329.0,175.0,95.13,4.87,310.0,-3.73
21,2022-12-06,136.0,59.0,FEDERALBNK.NS,21.52,1.79,0.78,138.0,81.0,96.67,3.33,149.0,9.56
59,2022-12-06,4432.0,82.0,BRITANNIA.NS,19.06,1.59,2.04,4433.0,3094.0,99.89,0.11,3935.0,-11.21
6,2022-12-06,897.0,65.0,AXISBANK.NS,17.16,1.70,1.42,915.0,626.0,93.93,6.07,1060.0,18.17
15,2022-12-06,616.0,66.0,SBIN.NS,16.51,1.51,1.03,616.0,435.0,100.19,-0.19,710.0,15.26
10,2022-12-06,1195.0,59.0,INDUSINDBK.NS,14.28,1.34,0.52,1264.0,778.0,85.88,14.12,1393.0,16.57
42,2022-12-06,337.0,41.0,ITC.NS,12.42,1.33,0.29,361.0,208.0,84.48,15.52,350.0,3.86
46,2022-12-06,1024.0,53.0,SUNPHARMA.NS,12.30,1.43,1.11,1052.0,737.0,91.16,8.84,1056.0,3.12
54,2022-12-06,2078.0,63.0,LT.NS,12.25,1.24,1.11,2104.0,1472.0,95.85,4.15,2036.0,-2.02
40,2022-12-06,838.0,54.0,BHARTIARTL.NS,11.67,1.20,0.86,851.0,641.0,93.60,6.40,879.0,4.89


In [87]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-06,669.0,41.0,MOTILALOFS.NS,-13.66,-1.55,-1.01,980.0,654.0,4.65,95.35,930.0,39.01
1,2022-12-06,463.0,72.0,GODREJIND.NS,-0.86,-0.64,1.15,644.0,405.0,24.27,75.73,638.0,37.80
2,2022-12-06,473.0,43.0,ICICIPRULI.NS,-10.18,-1.14,-1.00,616.0,447.0,15.59,84.41,640.0,35.31
3,2022-12-06,1287.0,61.0,SBILIFE.NS,6.58,0.50,0.56,1332.0,1025.0,85.49,14.51,1604.0,24.63
11,2022-12-06,586.0,67.0,HDFCLIFE.NS,3.17,0.35,1.45,694.0,504.0,43.16,56.84,681.0,16.21
14,2022-12-06,4057.0,56.0,NAUKRI.NS,-4.30,-0.71,0.25,5740.0,3375.0,28.82,71.18,4677.0,15.28
25,2022-12-06,1121.0,61.0,MUTHOOTFIN.NS,-1.90,-0.52,0.84,1540.0,956.0,28.30,71.70,1217.0,8.56
31,2022-12-06,1085.0,56.0,TECHM.NS,-6.70,-0.77,0.66,1806.0,965.0,14.27,85.73,1158.0,6.73
35,2022-12-06,1616.0,58.0,INFY.NS,3.32,0.45,1.23,1940.0,1365.0,43.65,56.35,1715.0,6.13
36,2022-12-06,3360.0,44.0,DIVISLAB.NS,-11.76,-1.23,-0.63,4678.0,3266.0,6.66,93.34,3560.0,5.95


In [88]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     19.703015
1     19.091873
2     17.834234
3     12.440022
11     8.187282
14     7.717561
25     4.323451
31     3.399162
35     3.096116
36     3.005202
48     1.111167
50     0.136371
51    -0.045457
Name: Upside%, dtype: float64

In [89]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)].sort_values(by = 'Upside%', ascending=False)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
4,2022-12-06,6707.0,37.0,BAJFINANCE.NS,-2.22,-0.20,-0.82,7853.0,5284.0,55.41,44.59,8015.0,19.50
5,2022-12-06,923.0,51.0,ICICIBANK.NS,11.64,1.33,0.77,953.0,654.0,90.07,9.93,1094.0,18.53
6,2022-12-06,897.0,65.0,AXISBANK.NS,17.16,1.70,1.42,915.0,626.0,93.93,6.07,1060.0,18.17
7,2022-12-06,551.0,42.0,JUBLFOOD.NS,-7.02,-0.84,-1.04,786.0,464.0,27.13,72.87,649.0,17.79
8,2022-12-06,3290.0,33.0,EICHERMOT.NS,4.91,0.86,-0.60,3850.0,2256.0,64.87,35.13,3865.0,17.48
9,2022-12-06,1256.0,46.0,M&M.NS,11.21,1.15,0.08,1354.0,691.0,85.16,14.84,1468.0,16.88
10,2022-12-06,1195.0,59.0,INDUSINDBK.NS,14.28,1.34,0.52,1264.0,778.0,85.88,14.12,1393.0,16.57
12,2022-12-06,78.0,63.0,NATIONALUM.NS,-5.72,-1.20,1.33,130.0,67.0,17.86,82.14,90.0,15.38
13,2022-12-06,429.0,52.0,TATAMOTORS.NS,-1.50,-0.23,0.47,525.0,372.0,37.55,62.45,495.0,15.38
15,2022-12-06,616.0,66.0,SBIN.NS,16.51,1.51,1.03,616.0,435.0,100.19,-0.19,710.0,15.26


In [90]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)].sort_values(by = 'Dev%_200', ascending=True)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
36,2022-12-06,3360.0,44.0,DIVISLAB.NS,-11.76,-1.23,-0.63,4678.0,3266.0,6.66,93.34,3560.0,5.95
44,2022-12-06,410.0,64.0,WIPRO.NS,-9.78,-1.32,0.50,722.0,375.0,10.23,89.77,425.0,3.66
7,2022-12-06,551.0,42.0,JUBLFOOD.NS,-7.02,-0.84,-1.04,786.0,464.0,27.13,72.87,649.0,17.79
31,2022-12-06,1085.0,56.0,TECHM.NS,-6.70,-0.77,0.66,1806.0,965.0,14.27,85.73,1158.0,6.73
39,2022-12-06,619.0,55.0,BERGEPAINT.NS,-3.70,-0.31,0.16,791.0,561.0,25.30,74.70,651.0,5.17
24,2022-12-06,1223.0,45.0,HAVELLS.NS,-2.87,-0.16,-0.08,1418.0,1070.0,43.94,56.06,1328.0,8.59
47,2022-11-22,3434.0,47.0,MINDTREE.NS,-1.89,0.17,0.62,4842.0,2742.0,32.95,67.05,3520.0,2.50
57,2022-11-29,4912.0,54.0,LTI.NS,-1.56,-0.22,0.57,7562.0,3824.0,29.11,70.89,4615.0,-6.05
32,2022-12-06,3155.0,53.0,ASIANPAINT.NS,0.11,-0.03,-0.06,3576.0,2580.0,57.77,42.23,3361.0,6.53
50,2022-12-06,3376.0,57.0,TCS.NS,1.61,0.21,1.04,4019.0,2982.0,37.96,62.04,3385.0,0.27


In [91]:
# turning around 
df_prec_dev[(df_prec_dev['Slope%_200'] > 0) & (df_prec_dev['Slope%_200'] < 1)].sort_values(by = 'Slope%_200', ascending=True)

,Date,Close Price,RSI,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
26,2022-12-06,884.0,66.0,GODREJCP.NS,4.23,0.12,0.58,968.0,665.0,72.23,27.77,958.0,8.37
60,2022-12-06,734.0,52.0,IRCTC.NS,2.07,0.14,0.14,903.0,571.0,49.07,50.93,571.0,-22.21
47,2022-11-22,3434.0,47.0,MINDTREE.NS,-1.89,0.17,0.62,4842.0,2742.0,32.95,67.05,3520.0,2.50
38,2022-12-06,2242.0,59.0,DEEPAKNTR.NS,6.13,0.19,0.32,2660.0,1720.0,55.59,44.41,2371.0,5.75
50,2022-12-06,3376.0,57.0,TCS.NS,1.61,0.21,1.04,4019.0,2982.0,37.96,62.04,3385.0,0.27
22,2022-12-06,4410.0,48.0,DRREDDY.NS,1.67,0.25,0.34,4907.0,3719.0,58.19,41.81,4829.0,9.50
23,2022-12-06,796.0,55.0,TATACONSUM.NS,3.10,0.26,0.64,849.0,660.0,71.93,28.07,871.0,9.42
49,2022-12-06,1599.0,52.0,COLPAL.NS,2.58,0.32,0.25,1678.0,1394.0,72.25,27.75,1606.0,0.44
19,2022-12-06,1932.0,65.0,INDIGO.NS,4.55,0.32,1.03,2265.0,1549.0,53.49,46.51,2176.0,12.63
11,2022-12-06,586.0,67.0,HDFCLIFE.NS,3.17,0.35,1.45,694.0,504.0,43.16,56.84,681.0,16.21
